## Import Synapse ML

In [ ]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

## Query Data

In [ ]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")
display(df)

## Configure Sentiment Analysis Model

In [ ]:
#Import the model and configure the input and output columns
model = (AnalyzeText()
    .setTextCol("description")
    .setKind("SentimentAnalysis") ## Can be changed to any othe rtext related purpose like detecting language
    .setOutputCol("response")
    .setErrorCol("error"))

## Apply Model

In [ ]:
#Apply the model to our dataframe
result = model.transform(df)
display(result)

## Create Sentiment Column

In [ ]:
#Create Sentiment Column
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

## Drop Columns (Original Description, Error) plus Display

In [ ]:
sentiment_df_final = sentiment_df.drop("error","response")
display(sentiment_df_final)

## Convert the date Published to Date Format

In [ ]:
from pyspark.sql.functions import col, to_date
sentiment_df_final = sentiment_df_final.withColumn("datePublished", to_date(col("datePublished"), "dd-MMM-yyyy"))

## Write to Delta Table with Merge Logic Type 1

In [ ]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_sentiment_analysis'
    
    sentiment_df_final.write.format("delta").saveAsTable(table_name)
    
except AnalysisException:
    print("Table Already Exists")
    
    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")
    
    spark.sql(f""" MERGE INTO {table_name} target_table
                  USING vw_sentiment_df_final source_view
                  ON source_view.url = target_table.url
                  
                  WHEN MATCHED AND
                  source_view.title <> target_table.title OR
                  source_view.description <> target_table.description OR
                  source_view.category <> target_table.category OR
                  source_view.image <> target_table.image OR
                  source_view.provider <> target_table.provider OR
                  source_view.datePublished <> target_table.datePublished
                  
                  THEN UPDATE SET *
                  
                  WHEN NOT MATCHED THEN INSERT *
              """)